In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '#ADD HER YOUR API KEY'

In [ ]:
!pip install litellm

import json
import os
from typing import List
from litellm import completion # OpenAI wrapper for calling LLMs.

In [ ]:
def list_files() -> List[str]:
    """List all files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Reads and returns the file content."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Assign names to Python functions, so the agent can call them.
tool_functions = {
    "list_files": list_files,
    "read_file": read_file
}

# Describe the functions in a way the LLM can understand.
tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    }
]

In [ ]:
# Set behavior guidelines to the model. Instruct the agent to list the files before attempting to read any.
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.
If a user asks about files, documents, or content, first list the files before reading them.
"""
}]

# Get input task from user.
user_task = input("What would you like me to do? ")
memory = [{"role": "user", "content": user_task}]
messages = agent_rules + memory


# Send the prompt to the model.
response = completion(
    model="openai/gpt-4o",
    messages=messages,
    tools=tools,
    max_tokens=1024
)

In [ ]:
tool = response.choices[0].message.tool_calls[0]  # Extracts which tool the model wants to use.
tool_name = tool.function.name
tool_args = json.loads(tool.function.arguments)
result = tool_functions[tool_name](**tool_args)   # Calls the corresponding Python function with the arguments.

print(f"Tool Name: {tool_name}")
print(f"Tool Arguments: {tool_args}")
print(f"Result: {result}")